In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#   SET UP
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


# Dash packages
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input, State, ALL
import plotly.graph_objects as go
import dash_leaflet as dl
from dash_leaflet import express as dlx
import kpi_heatmaps as kh


# Data packages
import pandas as pd
import json


# Baixar JSON
with open('assets/sc_geo.json', 'r') as sc:
    geo = json.load(sc)
    
    
# KPIs
KPIS = list(geo['features'][0]['properties'].keys())[-4:]

UFS = ['AC - Acre',              'AL - Alagoas',              'AP - Amapá',
       'AM - Amazonas',          'BA - Bahia',                'CE - Ceará',
       'DF - Distrito Federal',  'ES - Espírito Santo',       'GO - Goiás',
       'MA - Maranhão',          'MT - Mato Grosso',          'MS - Mato Grosso do Sul',
       'MG - Minas Gerais',      'PA - Pará',                 'PB - Paraíba',
       'PR - Paraná',            'PE - Pernambuco',           'PI - Piauí',
       'RJ - Rio de Janeiro',    'RN - Rio Grande do Norte',  'RS - Rio Grande do Sul',
       'RO - Rondônia',          'RR - Roraima',              'SC - Santa Catarina',
       'SP - São Paulo',         'SE - Sergipe',              'TO - Tocantins']


marks = [0, 10, 20, 30, 60, 70, 80, 90]
colorscale = ['#FFEDA0', '#FED976', '#FEB24C', '#FD8D3C', '#FC4E2A', '#E31A1C', '#BD0026', '#800026']

#def get_lixo(feature):
#    color = [colorscale[i] for i, item in enumerate(marks) if float(feature["properties"]['Lixo Coletado']) > item][-1]
#    return dict(fillColor=color, weight=2, opacity=1, color='white', dashArray='3', fillOpacity=0.7)

#def get_energia(feature):
#    color = [colorscale[i] for i, item in enumerate(marks) if float(feature["properties"]['Energia Elétrica']) > item][-1]
#    return dict(fillColor=color, weight=2, opacity=1, color='white', dashArray='3', fillOpacity=0.7)

#def get_agua(feature):
#    color = [colorscale[i] for i, item in enumerate(marks) if float(feature["properties"]['Rede Geral de Abastecimento de Água']) > item][-1]
#    return dict(fillColor=color, weight=2, opacity=1, color='white', dashArray='3', fillOpacity=0.7)

#def get_sanitario(feature):
#    color = [colorscale[i] for i, item in enumerate(marks) if float(feature["properties"]['Sem Banheiro ou Sanitário']) > item][-1]
#    return dict(fillColor=color, weight=2, opacity=1, color='white', dashArray='3', fillOpacity=0.7)

style_functions = {
    'Lixo Coletado': kh.get_lixo,
    'Energia Elétrica': kh.get_energia,
    'Rede Geral de Abastecimento de Água': kh.get_agua,
    'Sem Banheiro ou Sanitário': kh.get_sanitario
}


def get_info(feature=None, kpi=None):
    header = [html.H4("Santa Catarina")]
    if not feature:
        return header + ["Escolha um município"]
    return header + [html.B(feature["properties"]["name"]), html.Br(),
                     f'{kpi}: {float(feature["properties"][kpi])}%']


# Create app.
app = dash.Dash(external_stylesheets=[dbc.themes.SANDSTONE])
app.title = 'Mapa'
app.layout = html.Div([
    
    dcc.Store('memory_kpi', data='Lixo Coletado'),
    
    dbc.NavbarSimple([
        #dbc.DropdownMenu([dbc.DropdownMenuItem(uf, id={'item_uf': uf}) for uf in UFS],
        #                 direction='left',
        #                 nav=True,
        #                 in_navbar=True,
        #                 label='UF'),
        
        dbc.DropdownMenu([dbc.DropdownMenuItem(kpi, id={'item_kpi': kpi}) for kpi in KPIS],
                         direction='left',
                         nav=True,
                         in_navbar=True,
                         label='Indicador',
                         id='button_kpi')
    ],
    brand="Indicadores por Município",
    brand_href="#",
    color="primary",
    dark=True,
),
    
    dbc.Container([
        dbc.Row(style={'width': '100%', 'height': '90vh', 'margin': "auto", "display": "block"}, id="map")
    ])
    
])


@app.callback([Output('map', 'children'),
               Output('button_kpi', 'label'),
               Output('memory_kpi', 'data')],
             [Input({'item_kpi': ALL}, 'n_clicks')])
def set_kpi(click):
    
    kpi = dash.callback_context.triggered[0]['prop_id'][13:-11]
    
    # Create colorbar
    ctg = ["{}+".format(mark, marks[i + 1]) for i, mark in enumerate(marks[:-1])] + ["{}+".format(marks[-1])]
    colorbar = dlx.categorical_colorbar(categories=ctg, colorscale=colorscale, width=300, height=30, position="bottomleft")
    
    # Create geojson
    options = {'hoverStyle': {'weight': 5, 'color': '#666', 'dashArray': ''}, 'zoomToBoundsOnClick': True}
    geojson = dlx.geojson(geo, id="geojson", defaultOptions=options, style=style_functions[kpi])
    
    # Create info control
    info = html.Div(children=get_info(), id="info", className="info",
                    style={"position": "absolute", "bottom": "10px", "right": "10px", "z-index": "1000"})
    
    return dl.Map(children=[dl.TileLayer(), geojson, colorbar, info], center=[-27, -50], zoom=7), kpi, kpi


@app.callback(Output("info", "children"),
             [Input("geojson", "featureHover")],
             [State('memory_kpi', 'data')])
def info_hover(feature, data):
    return get_info(feature, kpi=data)
    
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Jul/2020 14:57:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 14:57:50] "GET /_dash-component-suites/dash_renderer/react@16.v1_5_1m1593643348.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 14:57:50] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_5_1m1593643348.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 14:57:50] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_5_1m1593643348.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 14:57:50] "GET /assets/style.css?m=1593649126.0682344 HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 14:57:50] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_5_1m1593643348.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 14:57:50] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_10_1m1593643360.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 14:57:50] "GET /_dash-component-suites/dash_bootstrap_

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\ProgramData\Anaconda3\lib\site-packages\dash\dash.py", line 1031, in dispatch
    response.set_d

127.0.0.1 - - [05/Jul/2020 14:57:50] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [05/Jul/2020 14:57:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 14:57:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 14:57:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 14:57:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 14:57:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 14:57:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 14:57:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 14:57:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 14:57:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 14:57:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 14:57:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.